---
title: "Multinomial Logit Model"
author: "Dominic Schenone"
date: "May 28 2025"
callout-appearance: minimal # this hides the blue "i" icon on .callout-notes
execute:
  warning: false
  message: false
---


This assignment expores two methods for estimating the MNL model: (1) via Maximum Likelihood, and (2) via a Bayesian approach using a Metropolis-Hastings MCMC algorithm. 


## 1. Likelihood for the Multi-nomial Logit (MNL) Model

Suppose we have $i=1,\ldots,n$ consumers who each select exactly one product $j$ from a set of $J$ products. The outcome variable is the identity of the product chosen $y_i \in \{1, \ldots, J\}$ or equivalently a vector of $J-1$ zeros and $1$ one, where the $1$ indicates the selected product. For example, if the third product was chosen out of 3 products, then either $y=3$ or $y=(0,0,1)$ depending on how we want to represent it. Suppose also that we have a vector of data on each product $x_j$ (eg, brand, price, etc.). 

We model the consumer's decision as the selection of the product that provides the most utility, and we'll specify the utility function as a linear function of the product characteristics:

$$ U_{ij} = x_j'\beta + \epsilon_{ij} $$

where $\epsilon_{ij}$ is an i.i.d. extreme value error term. 

The choice of the i.i.d. extreme value error term leads to a closed-form expression for the probability that consumer $i$ chooses product $j$:

$$ \mathbb{P}_i(j) = \frac{e^{x_j'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

For example, if there are 3 products, the probability that consumer $i$ chooses product 3 is:

$$ \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{e^{x_1'\beta} + e^{x_2'\beta} + e^{x_3'\beta}} $$

A clever way to write the individual likelihood function for consumer $i$ is the product of the $J$ probabilities, each raised to the power of an indicator variable ($\delta_{ij}$) that indicates the chosen product:

$$ L_i(\beta) = \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} = \mathbb{P}_i(1)^{\delta_{i1}} \times \ldots \times \mathbb{P}_i(J)^{\delta_{iJ}}$$

Notice that if the consumer selected product $j=3$, then $\delta_{i3}=1$ while $\delta_{i1}=\delta_{i2}=0$ and the likelihood is:

$$ L_i(\beta) = \mathbb{P}_i(1)^0 \times \mathbb{P}_i(2)^0 \times \mathbb{P}_i(3)^1 = \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{\sum_{k=1}^3e^{x_k'\beta}} $$

The joint likelihood (across all consumers) is the product of the $n$ individual likelihoods:

$$ L_n(\beta) = \prod_{i=1}^n L_i(\beta) = \prod_{i=1}^n \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} $$

And the joint log-likelihood function is:

$$ \ell_n(\beta) = \sum_{i=1}^n \sum_{j=1}^J \delta_{ij} \log(\mathbb{P}_i(j)) $$



## 2. Simulate Conjoint Data

We will simulate data from a conjoint experiment about video content streaming services. We elect to simulate 100 respondents, each completing 10 choice tasks, where they choose from three alternatives per task. For simplicity, there is not a "no choice" option; each simulated respondent must select one of the 3 alternatives. 

Each alternative is a hypothetical streaming offer consistent of three attributes: (1) brand is either Netflix, Amazon Prime, or Hulu; (2) ads can either be part of the experience, or it can be ad-free, and (3) price per month ranges from \$4 to \$32 in increments of \$4.

The part-worths (ie, preference weights or beta parameters) for the attribute levels will be 1.0 for Netflix, 0.5 for Amazon Prime (with 0 for Hulu as the reference brand); -0.8 for included adverstisements (0 for ad-free); and -0.1*price so that utility to consumer $i$ for hypothethical streaming service $j$ is 

$$
u_{ij} = (1 \times Netflix_j) + (0.5 \times Prime_j) + (-0.8*Ads_j) - 0.1\times Price_j + \varepsilon_{ij}
$$

where the variables are binary indicators and $\varepsilon$ is Type 1 Extreme Value (ie, Gumble) distributed.

The following code provides the simulation of the conjoint data.

:::: {.callout-note collapse="true"}

In [ ]:
import pandas as pd
import numpy as np

# Set seed for reproducibility
np.random.seed(123)

# Define attributes
brands = ["N", "P", "H"]  # Netflix, Prime, Hulu
ads = ["Yes", "No"]
prices = np.arange(8, 33, 4)

# Generate all possible profiles
profiles = pd.DataFrame([(b, a, p) for b in brands for a in ads for p in prices],
                        columns=["brand", "ad", "price"])

# Utility functions
b_util = {"N": 1.0, "P": 0.5, "H": 0.0}
a_util = {"Yes": -0.8, "No": 0.0}
p_util = lambda p: -0.1 * p

# Simulation parameters
n_peeps = 100
n_tasks = 10
n_alts = 3

# Simulate one respondent
def sim_one(id):
    datlist = []
    for t in range(1, n_tasks + 1):
        sampled = profiles.sample(n=n_alts, replace=False).copy()
        sampled.insert(0, "task", t)
        sampled.insert(0, "resp", id)
        sampled["v"] = (
            sampled["brand"].map(b_util) +
            sampled["ad"].map(a_util) +
            p_util(sampled["price"])
        ).round(10)
        sampled["e"] = -np.log(-np.log(np.random.rand(n_alts)))
        sampled["u"] = sampled["v"] + sampled["e"]
        sampled["choice"] = (sampled["u"] == sampled["u"].max()).astype(int)
        datlist.append(sampled)
    return pd.concat(datlist, ignore_index=True)

# Generate data
conjoint_data = pd.concat([sim_one(i) for i in range(1, n_peeps + 1)], ignore_index=True)

# Keep only observable variables
conjoint_data = conjoint_data[["resp", "task", "brand", "ad", "price", "choice"]]

::::



## 3. Preparing the Data for Estimation

The "hard part" of the MNL likelihood function is organizing the data, as we need to keep track of 3 dimensions (consumer $i$, covariate $k$, and product $j$) instead of the typical 2 dimensions for cross-sectional regression models (consumer $i$ and covariate $k$). The fact that each task for each respondent has the same number of alternatives (3) helps.  In addition, we need to convert the categorical variables for brand and ads into binary variables.


To prepare the data:
- We created **dummy variables** for the `brand` attribute. Specifically, we included binary indicators for **Netflix** and **Prime Video**, using **Hulu** as the baseline.
- The `ad` feature was converted into a **binary variable**, where 1 indicates the presence of ads and 0 indicates an ad-free option.
- We retained the `price` and `choice` columns in their original form.

Below is the Python code used for data preprocessing. The previewed table shows that each row corresponds to one product alternative shown in a choice task.


In [ ]:
import pandas as pd

# Loading and and preprocessing of  data
df = pd.read_csv('conjoint_data.csv')
df = pd.get_dummies(df, columns=["brand"], drop_first=True)
df["ad"] = df["ad"].map({"Yes": 1, "No": 0})
df.rename(columns={"brand_N": "netflix", "brand_P": "prime"}, inplace=True)
df.head()

# Create design matrix and response vector

X_columns = ["netflix", "prime", "ad", "price"]
X = df[X_columns].values
y = df["choice"].values
df["choice_set"] = df["resp"].astype(str) + "_" + df["task"].astype(str)
choice_sets = df["choice_set"].values

## 4. Estimation via Maximum Likelihood

We begin by defining the log-likelihood function for the Multinomial Logit (MNL) model. For each choice set (i.e., a group of three product alternatives shown to a respondent), we compute the probability of the selected alternative using the softmax function applied to the utility values. These utilities are linear combinations of the feature values and coefficients (betas).

The function below returns the **negative log-likelihood**, which we will minimize using `scipy.optimize`.

In [ ]:
import numpy as np

def log_likelihood(beta, X, y, groups):
    log_lik = 0
    beta = np.array(beta)
    for group in np.unique(groups):
        idx = groups == group
        X_group = X[idx]
        y_group = y[idx]

        utilities = np.dot(X_group, beta)
        utilities = np.asarray(utilities, dtype=np.float64)  # <- Force to numpy float64

        exp_utilities = np.exp(utilities - np.max(utilities))  # numerical stability
        probs = exp_utilities / np.sum(exp_utilities)

        log_lik += np.log(probs[y_group == 1][0])

    return -log_lik


In [ ]:
from scipy.optimize import minimize
import numpy as np

# Start with zero coefficients
initial_beta = np.zeros(X.shape[1])

# Minimize the negative log-likelihood
result = minimize(
    log_likelihood,
    initial_beta,
    args=(X, y, choice_sets),
    method="BFGS",
    options={"disp": True}
)

# Estimated coefficients
beta_hat = result.x

# Standard errors from inverse Hessian
hessian_inv = result.hess_inv
se = np.sqrt(np.diag(hessian_inv))

# 95% confidence intervals
z = 1.96
conf_int = np.array([beta_hat - z * se, beta_hat + z * se]).T

# Combining everything into a table
import pandas as pd
summary_df = pd.DataFrame({
    "Estimate": beta_hat,
    "Std. Error": se,
    "95% CI Lower": conf_int[:, 0],
    "95% CI Upper": conf_int[:, 1]
}, index=["netflix", "prime", "ad", "price"])


We estimated the parameters of the multinomial logit model using the BFGS algorithm via `scipy.optimize.minimize()`, minimizing the negative log-likelihood function defined earlier. The model includes binary indicators for `netflix`, `prime`, and `ad`, with `hulu` and `no ads` as the respective baseline categories, along with the continuous variable `price`.

The table below summarizes the parameter estimates, standard errors (from the inverse Hessian), and 95% confidence intervals:

In [ ]:
summary_df.round(3)

::: {.callout-note icon="🧠"}

Interpreting the Estimates

- Brand Preference: Netflix and Prime are both preferred over the baseline (Hulu), with Netflix receiving the highest utility.

- Ad Dislike: The negative coefficient for ad confirms that respondents significantly prefer ad-free experiences.

- Price Sensitivity: The negative and significant coefficient for price aligns with economic intuition — as price increases, utility decreases.
:::


## 5. Estimation via Bayesian Methods

_todo: code up a metropolis-hasting MCMC sampler of the posterior distribution. Take 11,000 steps and throw away the first 1,000, retaining the subsequent 10,000._

In [ ]:
import numpy as np

# Log-prior function
def log_prior(beta):
    # Priors: N(0,5) for all except price (N(0,1))
    prior = -0.5 * ((beta[0]/np.sqrt(5))**2 +
                    (beta[1]/np.sqrt(5))**2 +
                    (beta[2]/np.sqrt(5))**2 +
                    (beta[3]/1)**2)
    return prior

# Log-posterior function
def log_posterior(beta, X, y, groups):
    return -log_likelihood(beta, X, y, groups) + log_prior(beta)

# MCMC settings
n_steps = 11000
burn_in = 1000
n_params = X.shape[1]

# Proposal distribution: diagonal covariances
proposal_sds = [0.05, 0.05, 0.05, 0.005]

# Storage
samples = np.zeros((n_steps, n_params))
beta_current = np.zeros(n_params)
log_post_current = log_posterior(beta_current, X, y, choice_sets)

# Metropolis-Hastings loop
for step in range(n_steps):
    # Propose new beta from independent normal perturbations
    beta_proposal = beta_current + np.random.normal(0, proposal_sds)

    # Compute log-posterior
    log_post_proposal = log_posterior(beta_proposal, X, y, choice_sets)

    # Acceptance probability
    accept_prob = min(1, np.exp(log_post_proposal - log_post_current))

    # Accept or reject
    if np.random.rand() < accept_prob:
        beta_current = beta_proposal
        log_post_current = log_post_proposal

    samples[step] = beta_current

_hint: Use N(0,5) priors for the betas on the binary variables, and a N(0,1) prior for the price beta._

_hint: instead of calculating post=lik*prior, you can work in the log-space and calculate log-post = log-lik + log-prior (this should enable you to re-use your log-likelihood function from the MLE section just above)_

_hint: King Markov (in the video) use a candidate distribution of a coin flip to decide whether to move left or right among his islands.  Unlike King Markov, we have 4 dimensions (because we have 4 betas) and our dimensions are continuous.  So, use a multivariate normal distribution to pospose the next location for the algorithm to move to. I recommend a MNV(mu, Sigma) where mu=c(0,0,0,0) and sigma has diagonal values c(0.05, 0.05, 0.05, 0.005) and zeros on the off-diagonal.  Since this MVN has no covariances, you can sample each dimension independently (so 4 univariate normals instead of 1 multivariate normal), where the first 3 univariate normals are N(0,0.05) and the last one if N(0,0.005)._


_todo: for at least one of the 4 parameters, show the trace plot of the algorithm, as well as the histogram of the posterior distribution._

_todo: report the 4 posterior means, standard deviations, and 95% credible intervals and compare them to your results from the Maximum Likelihood approach._



## 6. Discussion

_todo: Suppose you did not simulate the data. What do you observe about the parameter estimates? What does $\beta_\text{Netflix} > \beta_\text{Prime}$ mean? Does it make sense that $\beta_\text{price}$ is negative?_

_todo: At a high level, discuss what change you would need to make in order to simulate data from --- and estimate the parameters of --- a multi-level (aka random-parameter or hierarchical) model. This is the model we use to analyze "real world" conjoint data._
